In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import os
import folium.plugins

Primeiro dataset, accidentes xeográficos. De aquí imos coller a data, o tipo (terremoto, tsunami, explosión normal ou nuclear), a súa magnitude e coordenadas

In [7]:
url = 'X:/Mapa'
df = pd.read_csv(f'{url}/database.csv')
df

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,...,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,...,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,...,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,...,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


In [69]:
df = df[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Type','ID']]


Segundo dataset, tsunamis. De aquí imos coller o ano (non existe data máis precisa), as coordenadas e a magnitude

In [44]:
df2 = pd.read_csv(f'{url}/tsunamis.csv')
df2 = df2[['Year','Magnitude','Latitude','Longitude']]
df2.dropna(inplace=True)
df2

,Year,Magnitude,Latitude,Longitude
0,1950,7.7,11.000,-85.000
1,1950,7.5,14.310,-91.917
2,1950,7.5,16.480,-98.220
3,1951,6.9,19.500,-156.000
4,1951,7.3,23.800,121.700
...,...,...,...,...
404,1995,6.5,38.401,22.283
405,1995,6.8,28.094,130.148
406,1998,7.0,-2.943,142.582
407,2003,7.2,-45.104,167.144


Terceiro dataset, volcáns. De aquí imos coller a data, o nome do volcán, as coordenadas e a magnitude (representada co índice de explosividad volcánica)

In [37]:
df3 = pd.read_csv(f'{url}/volcanes.csv', sep=';')
df3.columns
df3 = df3[['Year','Month','Day','Volcano Name','Coordinates','Volcanic Explosivity Index']]
df3['Month'].fillna('--', inplace=True)
df3['Day'].fillna('--', inplace=True)
df3['Volcanic Explosivity Index'].fillna('-1.0', inplace=True)
df3

,Year,Month,Day,Volcano Name,Coordinates,Volcanic Explosivity Index
0,1280,--,--,Quilotoa,"-0.85,-78.9",6.0
1,1540,--,--,Augustine,"59.37,-153.42",4.0
2,1660,--,--,Long Island,"-5.358,147.12",6.0
3,1858,4.0,23.0,Tate-yama,"36.57,137.6",-1.0
4,1864,1.0,3.0,Kelut,"-7.93,112.308",2.0
...,...,...,...,...,...,...
830,2010,8.0,6.0,Karangetang [Api Siau],"2.78,125.48",3.0
831,2014,11.0,10.0,Kilauea,"19.425,-155.292",-1.0
832,2016,5.0,9.0,Sinabung,"3.17,98.392",-1.0
833,2019,1.0,7.0,Manam,"-4.1,145.061",4.0


Procedimento:<br>
· Cargamos o shapefile das placas tectónicas e o engadimos ao mapa de folium.<br>
· Engadimos os datasets ao mánager de clústeres.<br>
· Engadimos o mánager de clústeres ao mapa, e logo o descargamos como html.

In [74]:
m = folium.Map()

url2 = f"{url}/Placas/PB2002_plates.shp"
placas = gpd.read_file(url2)
folium.GeoJson(placas, name='Placas').add_to(m)
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

for i,u in df.iterrows():
    if str(u.Type) == 'Earthquake':
        tooltip = str(u.Magnitude)
        folium.Circle([u.Latitude, u.Longitude],radius=u.Magnitude*300,popup=u.Date,tooltip=tooltip,color='crimson').add_to(marker_cluster) 
    elif str(u.Type) == 'Nuclear Explosion':
        tooltip = str(u.Magnitude)
        folium.Circle([u.Latitude, u.Longitude],radius=u.Magnitude*300,popup=u.Date,tooltip=tooltip,color='green').add_to(marker_cluster)
    elif str(u.Type) == 'Explosion':
        tooltip = str(u.Magnitude)
        folium.Circle([u.Latitude, u.Longitude],radius=u.Magnitude*300,popup=u.Date,tooltip=tooltip,color='yellow').add_to(marker_cluster) 
    elif str(u.Type) == 'Rock Burst':
        tooltip = str(u.Magnitude)
        folium.Circle([u.Latitude, u.Longitude],radius=u.Magnitude*300,popup=u.Date,tooltip=tooltip,color='brown').add_to(marker_cluster) 
        
for i,u in df2.iterrows():
    tooltip = str(u.Magnitude)
    folium.Circle([u.Latitude,u.Longitude], radius = u.Magnitude*300,popup=str(u.Year),tooltip=tooltip,color='blue').add_to(marker_cluster)

for i,u in df3.iterrows():
    tooltip = str(u['Volcanic Explosivity Index'])
    folium.Circle([u.Coordinates.split(',')[0],u.Coordinates.split(',')[1]], 
                  radius = float(u['Volcanic Explosivity Index'])*1000,
                  popup=f'{str(u.Day)}/{str(u.Month)}/{str(u.Year)}',
                  tooltip=tooltip,
                  color='purple'
                 ).add_to(marker_cluster)
folium.LayerControl().add_to(m)  

m.save('X:/Mapa/output-mgv.html')